In [1]:
#| default_exp pdp_routes

In [1]:
#| export
import httpx
import pandas as pd
import io
from pprint import pprint

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

In [2]:
#| export
class PDP_NotRetrieved(de.DomoError):
    def __init__(self,
                 domo_instance,
                 function_name,
                 status,
                 message,
                 pdp_id = None,
                 ):

        super().__init__(
            domo_instance=domo_instance,
            entity_id=pdp_id,
            function_name=function_name,
            status=status,
            message=message
        )

In [3]:
#| export
async def get_pdp_policies(auth: dmda.DomoAuth, 
                           dataset_id: str, 
                           session: httpx.AsyncClient = None,
                           debug_api: bool = False) -> rgd.ResponseGetData:
    url = f'http://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups/'

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session=session
    )

    if len(res.response) == 0 or not res.is_success:
        raise PDP_NotRetrieved(
            domo_instance=auth.domo_instance,
            function_name='get_pdp_policies',
            status=res.status,
            message = 'failed to retrieve pdp policies')
    
    return res


####  PDP ####

#### generate_policy_parameter_simple & generate_policy_body used to generate body for create_policy endpoint

In [4]:
#| export
def generate_policy_parameter_simple(column_name: str, 
                                     column_values_list: list[str] = None, 
                                     operator='EQUALS', 
                                     ignore_case: bool = True):
    return {
        "type": "COLUMN",
        "name": column_name,
        "values": column_values_list,
        "operator": operator,
        "ignoreCase": ignore_case
    }

In [16]:
#| export
def generate_policy_body(policy_name: str, 
                         dataset_id: str, 
                         parameters_list: dict, #generated by generate_policy_parameter_simple method
                         policy_id: str = None, # only included if updating existing policy
                         user_ids: list[str] = None,
                         group_ids: list[str] = None, 
                         virtual_user_ids: list[str] = None):
    if not user_ids:
        user_ids = []

    if not group_ids:
        group_ids = []
    
    if not virtual_user_ids:
        virtual_user_ids = []

    body = {
        "name": policy_name,
        "dataSourceId": dataset_id,
        "userIds": user_ids,
        "virtualUserIds": virtual_user_ids,
        "groupIds": group_ids,
        "dataSourcePermissions": False,
        "parameters": [parameters_list]
    }

    if policy_id:
        body.update({'filterGroupId': policy_id})

    return body

#### sample implementation of generate_policy_parameter_simple and generate_policy_body

In [21]:
parameters = generate_policy_parameter_simple(
                column_name='Customer ID',
                column_values_list='Cust_ABC',
                operator='EQUALS',
                ignore_case=True
            )

body = generate_policy_body('Test Policy Name', 
                            'abcdataset123',
                            policy_id='123', # including the policy_id updates that existing policy
                            parameters_list=parameters)
pprint(body)

{'dataSourceId': 'abcdataset123',
 'dataSourcePermissions': False,
 'filterGroupId': '123',
 'groupIds': [],
 'name': 'Test Policy Name',
 'parameters': [{'ignoreCase': True,
                 'name': 'Customer ID',
                 'operator': 'EQUALS',
                 'type': 'COLUMN',
                 'values': 'Cust_ABC'}],
 'userIds': [],
 'virtualUserIds': []}


In [ ]:
#| export
async def create_policy(auth: dmda.DomoAuth, 
                        dataset_id: str, 
                        body: dict, # generated using generate_policy_parameter_simple & generate_policy_body
                        session: httpx.AsyncClient = None,
                        debug_api: bool = False) -> rgd.ResponseGetData:
    
    url = f'https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups'
    
    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='POST',
        body=body,
        debug_api=debug_api,
        session=session)

    return res

#### sample implementation of create_policy

In [22]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

KeyError: 'DOMO_DOJO_ACCESS_TOKEN'

In [ ]:
#| export
async def update_policy(auth: dmda.DomoFullAuth, 
                        dataset_id: str, 
                        filter_group_id: str, 
                        body: dict, # generated using generate_policy_parameter_simple & generate_policy_body
                        session: httpx.AsyncClient = None,
                        debug_api: bool = False) -> rgd.ResponseGetData:
    
    url = f'https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups/{filter_group_id}'
    
    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        body=body,
        debug_api=debug_api,
        session=session
    )

    return res